In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
sys.path.append('../')

import argparse
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import shuffle

from loglizer.models import InvariantsMiner, PCA, IsolationForest, OneClassSVM, LogClustering, LR, SVM
from loglizer import dataloader, preprocessing
from loglizer.utils import metrics

D:\Anaconda\envs\LogBert\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\LogBert\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\Anaconda\envs\LogBert\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
ouput_dir = "../output/no_shuffle_tbird_325/"
middle_dir = ""
log_file = "Thunderbird_20M.log"

(x_train, y_train), (x_test, y_test) = dataloader.load_data(ouput_dir, middle_dir, log_file, is_mapping=True)
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train)
x_test = feature_extractor.transform(x_test)

E:\logbert-main\loglizer\dataloader.py:286: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train).reshape(-1,1)
E:\logbert-main\loglizer\dataloader.py:292: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_normal = np.array(test_normal).reshape(-1,1)
E:\logbert-main\loglizer\dataloader.py:298: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when 

Train normal size: 23134
Train abnormal size: 9092
Total logkey(exclude 0:UNK) 1206
Test normal size: 38538
Test abnormal size: 13640
num_unk_event in test data: 0
====== Transformed train data summary ======
Train data shape: 32226-by-1093

====== Transformed test data summary ======
Test data shape: 52178-by-1093



In [6]:
%%time
print("="*20 + " Model: PCA " + "="*20)
for th in np.arange(1):
    print("theshold", th)
    model = PCA(n_components=0.8, threshold=1, c_alpha = 1.9600)
    model.fit(x_train)
    print('Train validation:')
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Test validation:')
    precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: PCA ====================
theshold 0
====== Model summary ======
n_components: 6
Project matrix shape: 1093-by-1093
SPE threshold: 1

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 9092, FP: 23112, TN: 22, FN: 0
Precision: 28.233%, recall: 100.000%, F1-measure: 44.033%

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 13640, FP: 38508, TN: 30, FN: 0
Precision: 26.156%, recall: 100.000%, F1-measure: 41.467%

CPU times: total: 10.1 s
Wall time: 18.8 s


In [5]:
%%time
print("="*20 + " Model: IsolationForest " + "="*20)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=19)
model.fit(x_train)
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: IsolationForest ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 151, FP: 409, TN: 22725, FN: 8941
Precision: 26.964, recall: 1.661, F1-measure: 3.129

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 225, FP: 716, TN: 37822, FN: 13415
Precision: 23.911, recall: 1.650, F1-measure: 3.086

CPU times: total: 48.4 s
Wall time: 54.2 s


In [12]:
%%time
print("="*20 + " Model: one class SVM " + "="*20)
model = OneClassSVM(kernel='rbf')
model.fit(x_train, y_train)

print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: one class SVM ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1265, FP: 6000, TN: 0, FN: 1735
Precision: 17.412, recall: 42.167, F1-measure: 24.647

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 9313, FP: 42396, TN: 0, FN: 12998
Precision: 18.010, recall: 41.742, F1-measure: 25.163

CPU times: user 5min 30s, sys: 18.6 ms, total: 5min 30s
Wall time: 5min 30s


In [3]:
%%time
print("="*20 + " Model: LogClustering " + "="*20)
max_dist = 0.3  # the threshold to stop the clustering process
anomaly_threshold = 0.3  # the threshold for anomaly detection
model = LogClustering(max_dist=max_dist, anomaly_threshold=anomaly_threshold)
model.fit(x_train[y_train == 0, :])  # Use only normal samples for training
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)


==================== Model: LogClustering ====================
====== Model summary ======
Starting offline clustering...
Processed 1000 instances.
Found 77 clusters offline.

Starting online clustering...
Processed 2000 instances.
Processed 4000 instances.
Processed 6000 instances.
Processed 8000 instances.
Processed 10000 instances.
Processed 12000 instances.
Processed 14000 instances.
Processed 16000 instances.
Processed 18000 instances.
Processed 20000 instances.
Processed 22000 instances.
Processed 23134 instances.
Found 165 clusters online.

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 3828, FP: 4, TN: 23130, FN: 5264
Precision: 99.896, recall: 42.103, F1-measure: 59.239

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 5944, FP: 70, TN: 38468, FN: 7696
Precision: 98.836, recall: 43.578, F1-measure: 60.486

CPU times: total: 5min 11s
Wall time: 5min 28s
